In [2]:
from typing import Optional, Union

import numpy as np
from scipy.sparse import issparse, spmatrix
from scipy.sparse.linalg import LinearOperator, svds
from sklearn.utils import check_array, check_random_state
from sklearn.utils.extmath import svd_flip

from anndata import AnnData

from .. import logging as logg
from ._utils import _get_mean_var
from .._utils import AnyRandom
from .. import settings


def pca(
    data: Union[AnnData, np.ndarray, spmatrix],
    n_comps: Optional[int] = None,
    zero_center: Optional[bool] = True,
    svd_solver: str = 'arpack',
    random_state: AnyRandom = 0,
    return_info: bool = False,
    use_highly_variable: Optional[bool] = None,
    dtype: str = 'float32',
    copy: bool = False,
    chunked: bool = False,
    chunk_size: Optional[int] = None,
) -> Union[AnnData, np.ndarray, spmatrix]:
    """\
    Principal component analysis [Pedregosa11]_.
    Computes PCA coordinates, loadings and variance decomposition.
    Uses the implementation of *scikit-learn* [Pedregosa11]_.
    .. versionchanged:: 1.5.0
        In previous versions, computing a PCA on a sparse matrix would make a dense copy of
        the array for mean centering.
        As of scanpy 1.5.0, mean centering is implicit.
        While results are extremely similar, they are not exactly the same.
        If you would like to reproduce the old results, pass a dense array.
    Parameters
    ----------
    data
        The (annotated) data matrix of shape `n_obs` × `n_vars`.
        Rows correspond to cells and columns to genes.
    n_comps
        Number of principal components to compute. Defaults to 50, or 1 - minimum
        dimension size of selected representation.
    zero_center
        If `True`, compute standard PCA from covariance matrix.
        If `False`, omit zero-centering variables
        (uses :class:`~sklearn.decomposition.TruncatedSVD`),
        which allows to handle sparse input efficiently.
        Passing `None` decides automatically based on sparseness of the data.
    svd_solver
        SVD solver to use:
        `'arpack'` (the default)
          for the ARPACK wrapper in SciPy (:func:`~scipy.sparse.linalg.svds`)
        `'randomized'`
          for the randomized algorithm due to Halko (2009).
        `'auto'`
          chooses automatically depending on the size of the problem.
        `'lobpcg'`
          An alternative SciPy solver.
        .. versionchanged:: 1.4.5
           Default value changed from `'auto'` to `'arpack'`.
        Efficient computation of the principal components of a sparse matrix
        currently only works with the `'arpack`' or `'lobpcg'` solvers.
    random_state
        Change to use different initial states for the optimization.
    return_info
        Only relevant when not passing an :class:`~anndata.AnnData`:
        see “**Returns**”.
    use_highly_variable
        Whether to use highly variable genes only, stored in
        `.var['highly_variable']`.
        By default uses them if they have been determined beforehand.
    dtype
        Numpy data type string to which to convert the result.
    copy
        If an :class:`~anndata.AnnData` is passed, determines whether a copy
        is returned. Is ignored otherwise.
    chunked
        If `True`, perform an incremental PCA on segments of `chunk_size`.
        The incremental PCA automatically zero centers and ignores settings of
        `random_seed` and `svd_solver`. If `False`, perform a full PCA.
    chunk_size
        Number of observations to include in each chunk.
        Required if `chunked=True` was passed.
    Returns
    -------
    X_pca : :class:`~scipy.sparse.spmatrix`, :class:`~numpy.ndarray`
        If `data` is array-like and `return_info=False` was passed,
        this function only returns `X_pca`…
    adata : anndata.AnnData
        …otherwise if `copy=True` it returns or else adds fields to `adata`:
        `.obsm['X_pca']`
             PCA representation of data.
        `.varm['PCs']`
             The principal components containing the loadings.
        `.uns['pca']['variance_ratio']`
             Ratio of explained variance.
        `.uns['pca']['variance']`
             Explained variance, equivalent to the eigenvalues of the
             covariance matrix.
    """
    start = logg.info(f'computing PCA')

    # chunked calculation is not randomized, anyways
    if svd_solver in {'auto', 'randomized'} and not chunked:
        logg.info(
            'Note that scikit-learn\'s randomized PCA might not be exactly '
            'reproducible across different computational platforms. For exact '
            'reproducibility, choose `svd_solver=\'arpack\'.`'
        )
    data_is_AnnData = isinstance(data, AnnData)
    if data_is_AnnData:
        adata = data.copy() if copy else data
    else:
        adata = AnnData(data)

    if use_highly_variable is True and 'highly_variable' not in adata.var.keys():
        raise ValueError(
            'Did not find adata.var[\'highly_variable\']. '
            'Either your data already only consists of highly-variable genes '
            'or consider running `pp.highly_variable_genes` first.'
        )
    if use_highly_variable is None:
        use_highly_variable = True if 'highly_variable' in adata.var.keys() else False
    if use_highly_variable:
        logg.info('    on highly variable genes')
    adata_comp = (
        adata[:, adata.var['highly_variable']] if use_highly_variable else adata
    )

    if n_comps is None:
        min_dim = min(adata_comp.n_vars, adata_comp.n_obs)
        if settings.N_PCS >= min_dim:
            n_comps = min_dim - 1
        else:
            n_comps = settings.N_PCS

    logg.info(f'    with n_comps={n_comps}')

    random_state = check_random_state(random_state)

    X = adata_comp.X

    if chunked:
        if not zero_center or random_state or svd_solver != 'arpack':
            logg.debug('Ignoring zero_center, random_state, svd_solver')

        from sklearn.decomposition import IncrementalPCA

        X_pca = np.zeros((X.shape[0], n_comps), X.dtype)

        pca_ = IncrementalPCA(n_components=n_comps)

        for chunk, _, _ in adata_comp.chunked_X(chunk_size):
            chunk = chunk.toarray() if issparse(chunk) else chunk
            pca_.partial_fit(chunk)

        for chunk, start, end in adata_comp.chunked_X(chunk_size):
            chunk = chunk.toarray() if issparse(chunk) else chunk
            X_pca[start:end] = pca_.transform(chunk)
    elif (not issparse(X) or svd_solver == "randomized") and zero_center:
        from sklearn.decomposition import PCA

        if issparse(X) and svd_solver == "randomized":
            # This  is for backwards compat. Better behaviour would be to either error or use arpack.
            logg.warning(
                "svd_solver 'randomized' does not work with sparse input. Densifying the array. "
                "This may take a very large amount of memory."
            )
            X = X.toarray()
        pca_ = PCA(
            n_components=n_comps, svd_solver=svd_solver, random_state=random_state
        )
        X_pca = pca_.fit_transform(X)
    elif issparse(X) and zero_center:
        from sklearn.decomposition import PCA

        if svd_solver == "auto":
            svd_solver = "arpack"
        if svd_solver not in {'lobpcg', 'arpack'}:
            raise ValueError(
                'svd_solver: {svd_solver} can not be used with sparse input.\n'
                'Use "arpack" (the default) or "lobpcg" instead.'
            )

        output = _pca_with_sparse(
            X, n_comps, solver=svd_solver, random_state=random_state
        )
        # this is just a wrapper for the results
        X_pca = output['X_pca']
        pca_ = PCA(n_components=n_comps, svd_solver=svd_solver)
        pca_.components_ = output['components']
        pca_.explained_variance_ = output['variance']
        pca_.explained_variance_ratio_ = output['variance_ratio']
    elif not zero_center:
        from sklearn.decomposition import TruncatedSVD

        logg.debug(
            '    without zero-centering: \n'
            '    the explained variance does not correspond to the exact statistical defintion\n'
            '    the first component, e.g., might be heavily influenced by different means\n'
            '    the following components often resemble the exact PCA very closely'
        )
        pca_ = TruncatedSVD(
            n_components=n_comps, random_state=random_state, algorithm=svd_solver
        )
        X_pca = pca_.fit_transform(X)
    else:
        raise Exception("This shouldn't happen. Please open a bug report.")

    if X_pca.dtype.descr != np.dtype(dtype).descr:
        X_pca = X_pca.astype(dtype)

    if data_is_AnnData:
        adata.obsm['X_pca'] = X_pca
        adata.uns['pca'] = {}
        adata.uns['pca']['params'] = {
            'zero_center': zero_center,
            'use_highly_variable': use_highly_variable,
        }
        if use_highly_variable:
            adata.varm['PCs'] = np.zeros(shape=(adata.n_vars, n_comps))
            adata.varm['PCs'][adata.var['highly_variable']] = pca_.components_.T
        else:
            adata.varm['PCs'] = pca_.components_.T
        adata.uns['pca']['variance'] = pca_.explained_variance_
        adata.uns['pca']['variance_ratio'] = pca_.explained_variance_ratio_
        logg.info('    finished', time=start)
        logg.debug(
            'and added\n'
            '    \'X_pca\', the PCA coordinates (adata.obs)\n'
            '    \'PC1\', \'PC2\', ..., the loadings (adata.var)\n'
            '    \'pca_variance\', the variance / eigenvalues (adata.uns)\n'
            '    \'pca_variance_ratio\', the variance ratio (adata.uns)'
        )
        return adata if copy else None
    else:
        logg.info('    finished', time=start)
        if return_info:
            return (
                X_pca,
                pca_.components_,
                pca_.explained_variance_ratio_,
                pca_.explained_variance_,
            )
        else:
            return X_pca


def _pca_with_sparse(X, npcs, solver='arpack', mu=None, random_state=None):
    random_state = check_random_state(random_state)
    np.random.set_state(random_state.get_state())
    random_init = np.random.rand(np.min(X.shape))
    X = check_array(X, accept_sparse=['csr', 'csc'])

    if mu is None:
        mu = X.mean(0).A.flatten()[None, :]
    mdot = mu.dot
    mmat = mdot
    mhdot = mu.T.dot
    mhmat = mu.T.dot
    Xdot = X.dot
    Xmat = Xdot
    XHdot = X.T.conj().dot
    XHmat = XHdot
    ones = np.ones(X.shape[0])[None, :].dot

    def matvec(x):
        return Xdot(x) - mdot(x)

    def matmat(x):
        return Xmat(x) - mmat(x)

    def rmatvec(x):
        return XHdot(x) - mhdot(ones(x))

    def rmatmat(x):
        return XHmat(x) - mhmat(ones(x))

    XL = LinearOperator(
        matvec=matvec,
        dtype=X.dtype,
        matmat=matmat,
        shape=X.shape,
        rmatvec=rmatvec,
        rmatmat=rmatmat,
    )

    u, s, v = svds(XL, solver=solver, k=npcs, v0=random_init)
    u, v = svd_flip(u, v)
    idx = np.argsort(-s)
    v = v[idx, :]

    X_pca = (u * s)[:, idx]
    ev = s[idx] ** 2 / (X.shape[0] - 1)

    total_var = _get_mean_var(X)[1].sum()
    ev_ratio = ev / total_var

    output = {
        'X_pca': X_pca,
        'variance': ev,
        'variance_ratio': ev_ratio,
        'components': v,
    }
    return output

NameError: name 'AnyRandom' is not defined

In [26]:
from typing import Optional, Literal, Tuple, Union

from utils import gen_test_data

import numpy as np
import scipy.sparse as ss
import scipy.sparse.linalg as ssl
import sklearn.utils as su
import sklearn.metrics.pairwise as smp

In [6]:
X = gen_test_data()

random_state = None
random_state = su.check_random_state(random_state)
np.random.set_state(random_state.get_state())
random_init = np.random.rand(np.min(X.shape))

mu = X.mean(0).A.flatten()[None, :]
mdot = mu.dot
mmat = mdot
mhdot = mu.T.dot
mhmat = mu.T.dot
Xdot = X.dot
Xmat = Xdot
XHdot = X.T.conj().dot
XHmat = XHdot
ones = np.ones(X.shape[0])[None, :].dot

In [7]:
def matvec(x):
    return Xdot(x) - mdot(x)

def matmat(x):
    return Xmat(x) - mmat(x)

def rmatvec(x):
    return XHdot(x) - mhdot(ones(x))

def rmatmat(x):
    return XHmat(x) - mhmat(ones(x))

XL = LinearOperator(
    matvec=matvec,
    dtype=X.dtype,
    matmat=matmat,
    shape=X.shape,
    rmatvec=rmatvec,
    rmatmat=rmatmat,
)
u, s, v = svds(XL, solver='arpack', k=3, v0=random_init)

In [11]:
u

array([[-0.76460765, -0.31007793, -0.08696429],
       [ 0.06440737, -0.39592239,  0.303778  ],
       [ 0.38724412, -0.07124862, -0.8030008 ],
       [-0.16932712,  0.85764623,  0.08871865],
       [ 0.48228327, -0.08039728,  0.49746845]])

In [13]:
s

array([1.5075521 , 2.22831194, 3.68579836])

In [14]:
v

array([[ 0.33189392,  0.54958689, -0.29102236, -0.08252491,  0.47161172,
        -0.47630417, -0.00479207, -0.12125475,  0.17172498, -0.05289501],
       [-0.04131305, -0.17307923, -0.15273036, -0.31629146,  0.29856792,
         0.4604311 , -0.07247979, -0.03326805,  0.71036363,  0.18125613],
       [-0.28149533, -0.39407669,  0.01741264, -0.66254444,  0.26649959,
        -0.38419357, -0.15985804, -0.00564081, -0.28610132,  0.01133558]])

In [15]:
u, v = svd_flip(u, v)

In [16]:
u

array([[ 0.76460765, -0.31007793,  0.08696429],
       [-0.06440737, -0.39592239, -0.303778  ],
       [-0.38724412, -0.07124862,  0.8030008 ],
       [ 0.16932712,  0.85764623, -0.08871865],
       [-0.48228327, -0.08039728, -0.49746845]])

In [17]:
v

array([[-0.33189392, -0.54958689,  0.29102236,  0.08252491, -0.47161172,
         0.47630417,  0.00479207,  0.12125475, -0.17172498,  0.05289501],
       [-0.04131305, -0.17307923, -0.15273036, -0.31629146,  0.29856792,
         0.4604311 , -0.07247979, -0.03326805,  0.71036363,  0.18125613],
       [ 0.28149533,  0.39407669, -0.01741264,  0.66254444, -0.26649959,
         0.38419357,  0.15985804,  0.00564081,  0.28610132, -0.01133558]])

In [32]:
X_pca = (u * s)[:, idx]

In [33]:
idx = np.argsort(-s)

In [34]:
v = v[idx, :]

In [35]:
v

array([[-0.33189392, -0.54958689,  0.29102236,  0.08252491, -0.47161172,
         0.47630417,  0.00479207,  0.12125475, -0.17172498,  0.05289501],
       [-0.04131305, -0.17307923, -0.15273036, -0.31629146,  0.29856792,
         0.4604311 , -0.07247979, -0.03326805,  0.71036363,  0.18125613],
       [ 0.28149533,  0.39407669, -0.01741264,  0.66254444, -0.26649959,
         0.38419357,  0.15985804,  0.00564081,  0.28610132, -0.01133558]])

In [36]:
X_pca = (u * s)[:, idx]

In [37]:
X_pca

array([[ 0.32053283, -0.69095035,  1.15268587],
       [-1.11966445, -0.8822386 , -0.09709746],
       [ 2.95969904, -0.15876416, -0.58379069],
       [-0.32699904,  1.91110332,  0.25526945],
       [-1.83356838, -0.17915022, -0.72706716]])

In [38]:
ev = s[idx] ** 2 / (X.shape[0] - 1)

In [39]:
def my_get_mean_var(X, axis: Union[Literal['gene', 'cell'], int]):
    if not isinstance(axis, int):
        axis = 0 if axis == 'gene' else 1

    if isinstance(X, ss.spmatrix):  # same as sparse.issparse()
        X_copy = X.copy()
        X_copy.data **= 2
        mean = X.mean(axis=axis).A
        var = X_copy.mean(axis=axis).A - mean ** 2
        var *= X.shape[axis] / (X.shape[axis] - 1)
    else:
        mean = np.mean(X, axis=axis)
        var = np.var(X, axis=axis, ddof=1)  # a little overhead (mean counted twice, but it's ok.)
    return mean, var

In [40]:
my_get_mean_var(X, axis='cell')[1].sum()

2.4661045134208863

In [95]:
x = np.arange(10).reshape(10, 1)
x

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [101]:
mu.dot(x)

array([[18.99691169]])

In [103]:
mu @ x

array([[18.99691169]])

In [108]:
X.dot(x)

array([[17.11526048],
       [ 1.3768974 ],
       [36.12253499],
       [33.14591219],
       [ 7.2239534 ]])

In [112]:
mu @ x

array([[18.99691169]])

In [119]:
def matvec(x):
    return X @ x - mu @ x
#     return Xdot(x) - mdot(x)

def matmat(x):
    return X @ x - mu @ x
#     return X @ x - mu @ X.T

def rmatvec(x):
    return X.T.conj() @ x - mu.T @ np.ones(X.shape[0])[np.newaxis, :] @ x
#     return X.T.conj() @ x - mu.T @ (np.ones(X.shape[0])[None, :] @ x)
#     return XHdot(x) - mhdot(ones(x))

def rmatmat(x):
    return X.T.conj() @ x - mu.T @ np.ones(X.shape[0])[np.newaxis, :] @ x
#     return XHmat(x) - mhmat(ones(x))
    
XL = LinearOperator(
    matvec=matvec,
    dtype=X.dtype,
    matmat=matmat,
    shape=X.shape,
    rmatvec=rmatvec,
    rmatmat=rmatmat,
)
u, s, v = svds(XL, solver='arpack', k=3, v0=random_init)
s

array([1.5075521 , 2.22831194, 3.68579836])

In [84]:
s

array([1.5075521 , 2.22831194, 3.68579836])

In [120]:
X.toarray()

array([[0.        , 0.        , 0.60621461, 1.61436382, 0.        ,
        1.44374238, 0.36125158, 0.23907406, 0.        , 0.        ],
       [0.        , 0.60726616, 0.38481562, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.29207223, 2.03855503, 0.        , 2.86626514, 0.        ,
        1.81358986, 0.69463009, 0.        , 1.53118184, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.94502721,
        1.86892043, 0.        , 0.        , 1.97284884, 0.4709345 ],
       [0.        , 0.        , 0.        , 0.        , 1.80598835,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [134]:
Y = gen_test_data((5, 5))

In [135]:
Y.toarray()

array([[0.        , 0.        , 0.        , 0.59036004, 0.66271167],
       [1.17214564, 0.        , 0.        , 0.        , 0.88971561],
       [0.75820333, 0.        , 2.72971747, 1.22219577, 0.        ],
       [0.87893986, 0.84620373, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.93991671]])

In [138]:
ss.dia_matrix(Y + Y.T, 1).toarray()

array([[0.        , 1.17214564, 0.75820333, 1.4692999 , 0.66271167],
       [1.17214564, 0.        , 0.        , 0.84620373, 0.88971561],
       [0.75820333, 0.        , 5.45943494, 1.22219577, 0.        ],
       [1.4692999 , 0.84620373, 1.22219577, 0.        , 0.        ],
       [0.66271167, 0.88971561, 0.        , 0.        , 1.87983343]])

In [141]:
Y.eliminate_zeros()

In [143]:
Y.setdiag(1)

In [144]:
Y

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [145]:
Y.toarray()

array([[1.        , 0.        , 0.        , 0.59036004, 0.66271167],
       [1.17214564, 1.        , 0.        , 0.        , 0.88971561],
       [0.75820333, 0.        , 1.        , 1.22219577, 0.        ],
       [0.87893986, 0.84620373, 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])

In [147]:
import sklearn.metrics as skm

In [149]:
skm.r2_score(Y.toarray(), Y.toarray())

1.0

In [156]:
type((Y / Y.sum(axis=1)).A)

numpy.ndarray

In [155]:
np.corrcoef(Y.toarray(), Y.toarray())

array([[ 1.        ,  0.38265389,  0.13686109,  0.27411002,  0.26980894,
         1.        ,  0.38265389,  0.13686109,  0.27411002,  0.26980894],
       [ 0.38265389,  1.        , -0.72515244,  0.1824839 ,  0.27295455,
         0.38265389,  1.        , -0.72515244,  0.1824839 ,  0.27295455],
       [ 0.13686109, -0.72515244,  1.        ,  0.23205052, -0.58629324,
         0.13686109, -0.72515244,  1.        ,  0.23205052, -0.58629324],
       [ 0.27411002,  0.1824839 ,  0.23205052,  1.        , -0.60835327,
         0.27411002,  0.1824839 ,  0.23205052,  1.        , -0.60835327],
       [ 0.26980894,  0.27295455, -0.58629324, -0.60835327,  1.        ,
         0.26980894,  0.27295455, -0.58629324, -0.60835327,  1.        ],
       [ 1.        ,  0.38265389,  0.13686109,  0.27411002,  0.26980894,
         1.        ,  0.38265389,  0.13686109,  0.27411002,  0.26980894],
       [ 0.38265389,  1.        , -0.72515244,  0.1824839 ,  0.27295455,
         0.38265389,  1.        , -0.72515244

In [161]:
Y  = gen_test_data((1000, 1000))

In [162]:
Y

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 420837 stored elements in Compressed Sparse Row format>

In [163]:
Y @ Y

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

In [159]:
Y @Y @ Y @Y

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>